In [61]:
#!pip install -U tfx

# Inisiasi Variabel Directory

In [4]:
import os

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

DATA_ROOT   = 'data'
MODULE_PATH = 'module'

create_dir(DATA_ROOT)
create_dir(MODULE_PATH)

# Create Module
---

## Create Module Components

In [13]:
module_components = os.path.join(DATA_ROOT, 'components.py')

'data\\components.py'

In [14]:
%%writefile {module_components}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the components.py module.
Usage:
- COMPONENTS TFX PIPELINE
"""
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma

from tfx.components import (
    CsvExampleGen, 
    StatisticsGen, 
    SchemaGen, 
    ExampleValidator, 
    Transform, 
    Trainer, 
    Tuner, 
    Evaluator, 
    Pusher)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

def init_components(
    data_dir,
    transform_module,
    training_module,
    training_steps,
    eval_steps,
    serving_model_dir,
    tuner_module,
    tuner_steps 
):
    """
    Initializes components for a machine learning pipeline, optionally with hyperparameter tuning.

    Args:
        data_dir: Path to the directory containing training and evaluation data.
        transform_module: Module containing data transformation functions.
        training_module: Module containing training functions.
        training_steps: Number of training steps.
        eval_steps: Number of evaluation steps.
        serving_model_dir: Directory to save the serving model.
        tuner_module: Module containing tuner functions.
    """

    # Examplegen
    output = example_gen_pb2.Output(
        split_config = example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
            example_gen_pb2.SplitConfig.Split(name='eval',  hash_buckets=1)
        ])
    )

    example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

    # Statisticgen
    statistics_gen = StatisticsGen(
        examples = example_gen.outputs['examples']
    )

    # Schemagen
    schema_gen = SchemaGen(
        statistics = statistics_gen.outputs['statistics']
    )

    # Validator
    example_validator = ExampleValidator(
        statistics = statistics_gen.outputs['statistics'],
        schema     = schema_gen.outputs['schema']
    )

    # Transform
    transform = Transform(
        examples    = example_gen.outputs['examples'],
        schema      = schema_gen.outputs['schema'],
        module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
    )

    # Tuner
    tuner = Tuner(
        module_file     = os.path.abspath(TUNER_MODULE_FILE),
        examples        = transform.outputs['transformed_examples'],
        transform_graph = transform.outputs['transform_graph'],
        schema          = schema_gen.outputs['schema'],
        train_args      = trainer_pb2.TrainArgs(splits=['train']),
        eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
    )

    # Trainer
    trainer = Trainer(
        module_file     = os.path.abspath(TRAINER_MODULE_FILE),
        examples        = transform.outputs['transformed_examples'],
        transform_graph = transform.outputs['transform_graph'],
        schema          = schema_gen.outputs['schema'],
        hyperparameters = tuner.outputs['best_hyperparameters'],
        train_args      = trainer_pb2.TrainArgs(splits=['train']),
        eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
    )

    # Resolver
    model_resolver = Resolver(
        strategy_class = LatestBlessedModelStrategy,
        model          = Channel(type=Model),
        model_blessing = Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

    # Evaluator
    eval_config = tfma.EvalConfig(
        model_specs   = [tfma.ModelSpec(label_key = 'Category')],
        slicing_specs = [tfma.SlicingSpec()],
        metrics_specs = [
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name = 'ExampleCount'),
                tfma.MetricConfig(class_name = 'AUC'),
                tfma.MetricConfig(class_name = 'FalsePositives'),
                tfma.MetricConfig(class_name = 'TruePositives'),
                tfma.MetricConfig(class_name = 'FalseNegatives'),
                tfma.MetricConfig(class_name = 'TrueNegatives'),
                tfma.MetricConfig(class_name = 'BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold = tfma.GenericValueThreshold(
                            lower_bound = {'value': 0.5}
                        ),
                        change_threshold = tfma.GenericChangeThreshold(
                            direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute  = {'value': 0.0001}
                        )
                    )
                )
            ])
        ]
    )

    evaluator = Evaluator(
        examples       = example_gen.outputs['examples'],
        model          = trainer.outputs['model'],
        baseline_model = model_resolver.outputs['model'],
        eval_config    = eval_config
    )

    # Pusher
    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory='serving_model_dir/spam-email-model'
            )
        )
    )

    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )
    
    return components

Writing data\components.py


## Create Module Transform

In [ ]:
module_transform = os.path.join(DATA_ROOT, 'transform.py')

In [ ]:

%%writefile {TRANSFORM_MODULE_FILE}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the transform.py module.
Usage:
- Change text to int for tensorflow
"""

import tensorflow as tf

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

# Renaming transformed features
def transformed_name(key):
    return key + "_xf"

# Preprocess input features into transformed features
def preprocessing_fn(inputs):
    """
    inputs:  map from feature keys to raw features
    outputs: map from feature keys to transformed features
    """

    outputs = {}
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

## Create Module Tuner

In [ ]:
module_tuner = os.path.join(DATA_ROOT, 'tuner.py')

In [ ]:
%%writefile {module_tuner}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the tuner.py module.
Usage:
- For tuning best parameter ML
"""
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE      = 8000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim = hp.Int('embedding_dim', min_value=16, max_value=128, step=16)
    lstm_units    = hp.Int('lstm_units', min_value=16, max_value=128, step=16)
    num_layers    = hp.Choice('num_layers', values=[1, 2, 3])
    dense_units   = hp.Int('dense_units', min_value=16, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    for _ in range(num_layers):
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(learning_rate),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_binary_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2,
    min_delta= 0,
    baseline = 0.9,
    restore_best_weights =True
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set   = input_fn(fn_args.eval_files[0],  tf_transform_output, 10)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model tuner
    model_tuner = kt.Hyperband(
        hypermodel   = lambda hp: model_builder(hp),
        objective    = kt.Objective('val_binary_accuracy', direction='max'),
        max_epochs   = 5,
        factor       = 3,
        directory    = 'dhadhan-pipeline',
        project_name = 'spam_email_tuner',
    )

    model_tuner.oracle.max_trials = 3

    return TunerFnResult(
        tuner      = model_tuner,
        fit_kwargs = {
            'callbacks'        : [early_stop_callback],
            'x'                : train_set,
            'validation_data'  : val_set,
            'steps_per_epoch'  : fn_args.train_steps,
            'validation_steps' : fn_args.eval_steps
        }
    )

## Create Module Training

In [ ]:
module_train = os.path.join(DATA_ROOT, 'train.py')

In [ ]:
%%writefile {module_train}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the train.py module.
Usage:
- For training model
"""
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 8000
SEQUENCE_LENGTH = 100
embedding_dim   = 16

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    for _ in range(hp['num_layers']):
        x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback
    ]

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp['tuner/epochs'])
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, hp['tuner/epochs'])

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp['tuner/epochs']
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )

    plot_model(
        model, 
        to_file='images/model_plot.png', 
        show_shapes=True, 
        show_layer_names=True
    )

## Create Module local-pipeline

In [ ]:
module_pipeline = os.path.join(DATA_ROOT, 'local-pipeline.py')

In [ ]:
%%writefile {module_train}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the local-pipeline.py module.
Usage:
- For running Components module with pipeline orchestrator: Apache Beam.
"""
import os
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

PIPELINE_NAME = 'diabetes-pipeline'

DATA_ROOT = 'data'
TRANSFORM_MODULE_FILE = 'modules/transform.py'
TRAINER_MODULE_FILE = 'modules/trainer.py'

OUTPUT_BASE = 'output'
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')


def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    """
    Initialize a local TFX pipeline.

    Args:
        components: A dictionary of TFX components to be included in the pipeline.
        pipeline_root: Root directory for pipeline output artifacts.

    Returns:
        A TFX pipeline.
    """
    logging.info(f'Pipeline root set to: {pipeline_root}')
    beam_args = [
        '--direct_running_mode=multi_processing'
        # 0 auto-detect based on the number of CPUs available
        # duraing execution time
        '----direct_num_workers=0'
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )


if __name__ == '__main__':
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components
    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

# Load Data

In [3]:
os.environ['KAGGLE_USERNAME'] = 'ahrdadan'
os.environ['KAGGLE_KEY']      = '7eb307ba5bd108dae806763473bd568c'

In [64]:
!kaggle datasets download -d ashfakyeafi/spam-email-classification -f email.csv

Dataset URL: https://www.kaggle.com/datasets/ashfakyeafi/spam-email-classification
License(s): Apache 2.0



2024-05-01 03:00:34,621 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))': /api/v1/datasets/metadata/ashfakyeafi/spam-email-classification
2024-05-01 03:00:48,124 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))': /api/v1/datasets/metadata/ashfakyeafi/spam-email-classification
2024-05-01 03:00:51,585 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))': /api/v1/datasets/metadata/ashfakyeafi/spam-email-classification

  0%|          | 0.00/469k [00:00<?, ?B/s]
100%|██████████| 469k/469k [00:01<00:00, 437kB/s]
100%|██████████| 469k/469k [00:01<00:00, 436kB/s]


In [65]:
DATA_PATH = 'dataset'

DATASET_NAME = 'email.csv'

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df = pd.read_csv(DATASET_NAME)

df['Category'] = df['Category'].map({'spam': 1, 'ham': 0})
df['Category'] = df['Category'].fillna(0)
df['Category'] = df['Category'].astype(int)

df.to_csv(os.path.join(DATA_PATH, DATASET_NAME), index=False)
df = pd.read_csv(f'{DATA_PATH}/{DATASET_NAME}')

os.remove(DATASET_NAME)

df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives around here though"


# Pipeline
---


Initial interactive context and directory

In [66]:
PIPELINE_NAME = 'dhadhan-pipeline'
SCHEMA_PIPELINE_NAME = 'spam-email-tfdv-schema'

PIPELINE_ROOT = os.path.join(PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = DATA_PATH

context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

In [67]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval',  hash_buckets=1)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [68]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

In [69]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples']
)

context.run(statistics_gen)

  Using cached certifi-2022.9.24-py3-none-any.whl.metadata (2.9 kB)
Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle 1.6.12 requires certifi>=2023.7.22, but you have certifi 2022.9.24 which is incompatible.


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

### Buat data summary dengan statistic

In [70]:
context.show(statistics_gen.outputs["statistics"])

### Membuat data schema

In [71]:
schema_gen = SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [72]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Category',INT,required,,-
'Message',BYTES,required,,-


### Check Anomali pada dataset

In [73]:
example_validator = ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema     = schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [74]:
context.show(example_validator.outputs['anomalies'])

## Data Preprocessing
Data preprocessing dengan module transform

In [75]:
TRANSFORM_MODULE_FILE = 'spam_email_transform.py'

In [76]:

%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

# Renaming transformed features
def transformed_name(key):
    return key + "_xf"

# Preprocess input features into transformed features
def preprocessing_fn(inputs):
    """
    inputs:  map from feature keys to raw features
    outputs: map from feature keys to transformed features
    """

    outputs = {}
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    outputs[transformed_name(LABEL_KEY)]   = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Overwriting spam_email_transform.py


In [77]:

transform = Transform(
    examples    = example_gen.outputs['examples'],
    schema      = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

context.run(transform)

INFO:tensorflow:Assets written to: dhadhan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\cf7263c5cbf24c57a883dc0374a1f7eb\assets


INFO:tensorflow:Assets written to: dhadhan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\cf7263c5cbf24c57a883dc0374a1f7eb\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Tuning Model

In [78]:
TUNER_MODULE_FILE = 'spam_email_tuner.py'

In [85]:
%%writefile {TUNER_MODULE_FILE}
"""
Author: Ahmad Ramadhan
Date: 2024-05-02
This is the transform.py module.
Usage:
- COMPONENTS TFX PIPELINE
"""
import os
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from keras_tuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE      = 8000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build keras tuner model"""
    embedding_dim = hp.Int('embedding_dim', min_value=16, max_value=128, step=16)
    lstm_units    = hp.Int('lstm_units', min_value=16, max_value=128, step=16)
    num_layers    = hp.Choice('num_layers', values=[1, 2, 3])
    dense_units   = hp.Int('dense_units', min_value=16, max_value=128, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])

    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    for _ in range(num_layers):
        x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(learning_rate),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

TunerFnResult = NamedTuple('TunerFnResult', [
    ('tuner', base_tuner.BaseTuner),
    ('fit_kwargs', Dict[Text, Any]),
])

early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor  = 'val_binary_accuracy',
    mode     = 'max',
    verbose  = 1,
    patience = 2,
    min_delta= 0,
    baseline = 0.9,
    restore_best_weights =True
)

def tuner_fn(fn_args: FnArgs) -> None:
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, 10)
    val_set   = input_fn(fn_args.eval_files[0],  tf_transform_output, 10)

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model tuner
    model_tuner = kt.Hyperband(
        hypermodel   = lambda hp: model_builder(hp),
        objective    = kt.Objective('val_binary_accuracy', direction='max'),
        max_epochs   = 5,
        factor       = 3,
        directory    = 'dhadhan-pipeline',
        project_name = 'spam_email_tuner',
    )

    model_tuner.oracle.max_trials = 3

    return TunerFnResult(
        tuner      = model_tuner,
        fit_kwargs = {
            'callbacks'        : [early_stop_callback],
            'x'                : train_set,
            'validation_data'  : val_set,
            'steps_per_epoch'  : fn_args.train_steps,
            'validation_steps' : fn_args.eval_steps
        }
    )

Overwriting spam_email_tuner.py


In [86]:
tuner = Tuner(
    module_file     = os.path.abspath(TUNER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)


context.run(tuner)

Trial 3 Complete [00h 11m 48s]
val_binary_accuracy: 0.9817813634872437

Best val_binary_accuracy So Far: 0.9817813634872437
Total elapsed time: 00h 17m 58s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in dhadhan-pipeline\spam_email_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 32
lstm_units: 96
num_layers: 2
dense_units: 48
dropout_rate: 0.5
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9817813634872437
Trial summary
Hyperparameters:
embedding_dim: 64
lstm_units: 64
num_layers: 2
dense_units: 48
dropout_rate: 0.4
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.980431854724884
Trial summary
Hyperparameters:
embedding_dim: 64
lstm_units: 32
num_layers: 3
dense_units: 96
dropout_rate: 0.4
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.97773277759552


ExecutionResult(
    component_id: Tuner
    execution_id: 9
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## Model Training

In [87]:
TRAINER_MODULE_FILE = "spam_email_trainer.py"

In [88]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY   = "Category"
FEATURE_KEY = "Message"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern = file_pattern,
        batch_size   = batch_size,
        features     = transform_feature_spec,
        reader       = gzip_reader_fn,
        num_epochs   = num_epochs,
        label_key    = transformed_name(LABEL_KEY)
    )

    return dataset

# Vocabulary size and number of words in a sequence
VOCAB_SIZE      = 8000
SEQUENCE_LENGTH = 100
embedding_dim   = 16

vectorize_layer = layers.TextVectorization(
    standardize            = 'lower_and_strip_punctuation',
    max_tokens             = VOCAB_SIZE,
    output_mode            = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp['embedding_dim'], name='embedding')(x)
    x = layers.Bidirectional(layers.LSTM(hp['lstm_units']))(x)
    for _ in range(hp['num_layers']):
        x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout_rate'])(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    model.compile(
        loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer = tf.keras.optimizers.Adam(hp['learning_rate']),
        metrics   = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features      = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    hp      = fn_args.hyperparameters['values']

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor  = 'val_binary_accuracy',
        mode     = 'max',
        verbose  = 1,
        patience = 10
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor        = 'val_binary_accuracy',
        mode           = 'max',
        verbose        = 1,
        save_best_only = True
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callback,
        model_checkpoint_callback
    ]

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp['tuner/epochs'])
    val_set   = input_fn(fn_args.eval_files,  tf_transform_output, hp['tuner/epochs'])

    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)
        ]]
    )

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x                = train_set,
        validation_data  = val_set,
        callbacks        = callbacks,
        steps_per_epoch  = fn_args.train_steps,
        validation_steps = fn_args.eval_steps,
        epochs           = hp['tuner/epochs']
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(
                shape = [None],
                dtype = tf.string,
                name  = 'examples'
            )
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format = 'tf',
        signatures  = signatures
    )


Overwriting spam_email_trainer.py


In [89]:
trainer = Trainer(
    module_file     = os.path.abspath(TRAINER_MODULE_FILE),
    examples        = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval'])
)

context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Message_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 32)           256000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 192)              99072     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 48)                9264

INFO:tensorflow:Assets written to: dhadhan-pipeline\Trainer\model\10\Format-Serving\assets


INFO:tensorflow:Assets written to: dhadhan-pipeline\Trainer\model\10\Format-Serving\assets


128/128 [==============================] - 147s 1s/step - loss: 0.2508 - binary_accuracy: 0.9155 - val_loss: 0.0900 - val_binary_accuracy: 0.9717
Epoch 2/2
128/128 [==============================] - ETA: 0s - loss: 0.0703 - binary_accuracy: 0.9808
Epoch 2: val_binary_accuracy did not improve from 0.97166
128/128 [==============================] - 77s 603ms/step - loss: 0.0703 - binary_accuracy: 0.9808 - val_loss: 0.0896 - val_binary_accuracy: 0.9669
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: dhadhan-pipeline\Trainer\model\10\Format-Serving\assets


INFO:tensorflow:Assets written to: dhadhan-pipeline\Trainer\model\10\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Resolver

In [90]:
model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model          = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## Evaluator

In [91]:
eval_config = tfma.EvalConfig(
    model_specs   = [tfma.ModelSpec(label_key = 'Category')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name = 'ExampleCount'),
            tfma.MetricConfig(class_name = 'AUC'),
            tfma.MetricConfig(class_name = 'FalsePositives'),
            tfma.MetricConfig(class_name = 'TruePositives'),
            tfma.MetricConfig(class_name = 'FalseNegatives'),
            tfma.MetricConfig(class_name = 'TrueNegatives'),
            tfma.MetricConfig(class_name = 'BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold = tfma.GenericValueThreshold(
                        lower_bound = {'value': 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute  = {'value': 0.0001}
                    )
                )
            )
        ])
    ]
)

In [92]:
evaluator = Evaluator(
    examples       = example_gen.outputs['examples'],
    model          = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config    = eval_config
)

context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 12
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

### Visualisasi Evaluator

In [93]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Pusher
deploy ke tf serving

In [94]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/spam-email-model'))
 
)
 
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 13
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [95]:
!pip freeze > requirements.txt